In [108]:
import numpy as np
import pandas as pd
import gpstk
import matplotlib.pyplot as plt
%matplotlib inline

# Get stations baseline < 100 Km

In [109]:
data=pd.read_csv("conus_2003_324_324.csv")
data.columns=["Station", "a","b","c","x","y","z"]

In [110]:
data=data.drop(["a","b","c"],axis=1)
array=data.as_matrix()

In [111]:
def get_pairs(array):
    distances=[]
    for i in range(len(array)):
        for j in range(i+1,len(array)):
            distance=np.sqrt((array[i][1]-array[j][1])**2+(array[i][2]-array[j][2])**2+(array[i][3]-array[j][3])**2)
            if distance<100e3:
                distances.append([array[i][0],array[j][0]])
    return distances

In [112]:
pairs=get_pairs(array)
len(pairs)
pairs[:10]

[['1ulm', 'sihs'],
 ['1ulm', 'wnfl'],
 ['abq1', 'nmsf'],
 ['abq1', 'zab1'],
 ['acu1', 'npri'],
 ['adks', 'ang1'],
 ['adks', 'ang2'],
 ['adks', 'lkhu'],
 ['adks', 'netp'],
 ['adks', 'txhu']]

## Functions

In [113]:
def adjust_times(df):
    df=df.sort("TIME")
    times=df.TIME.as_matrix()
    for i in range(times.size):
        if times[i]%30!=0:
            times[i]=times[i]+(30-times[i]%30)
    df["TIME"]=times
    return df

In [114]:
def label_arcs(df):
    time=df.TIME.as_matrix()
    diff=np.diff(time)>3600
    diff=np.hstack((np.array([False]),diff))
    split=np.where(diff==True) #points where there is a "True" value
    #if split is none
    if len(split[0])!=0:
        arcs=np.split(time,split)
        arcsID=[]
        n=1
        for i in range(len(arcs)):
            size=len(arcs[i])
            tmp=np.empty(size)
            tmp[:]=n
            arcsID.append(tmp)
            n+=1
        arcsID=np.concatenate(arcsID)
    else:#one arc
        arcsID=np.empty(time.size)
        arcsID[:]=1
        
    df["ARCS"]=arcsID
    return df

In [115]:
def cycle_slips(PhaseDelay,threshold=0.5):
    slips=[]
    slips=np.where(np.abs(np.diff(np.hstack(([0],PhaseDelay))))>threshold) 
    return slips[0]
    

# Preprocessing of ten Pairs of stations


In [116]:
from os import listdir
dir_txt="txtcors/"
files = [f for f in listdir(dir_txt)]

f1=gpstk.L1_FREQ_GPS
f2=gpstk.L2_FREQ_GPS
factor_alfa=f2**2/(f1**2-f2**2)
c=3e8
alfa=1.0/((f1**2/f2**2)-1) 

for stations in pairs[:10]:
    #Load Files
    st1,st2=stations[0],stations[1]
    columns=["PRN","TIME","C1","C2","L1","L2","Tgd","IPP","Elevation"]
    file1=dir_txt+[f for f in files if st1 in f ][0]
    file2=dir_txt+[f for f in files if st2 in f ][0]
    df1=pd.read_csv(file1,sep=",")
    df1.columns=columns
    df2=pd.read_csv(file2,sep=",")
    df2.columns=columns
    df1,df2=adjust_times(df1),adjust_times(df2)
    df3=pd.merge(df1,df2,on=["TIME","PRN","Tgd"])
    
    for sat in np.unique(df3.PRN.values):
        
        df=df3[df3.PRN==sat] #dataframe with times of an specific satellite
        df=df.reset_index(drop=True)
        df["PhaseDelay_x"]=alfa*(df.L1_x-df.L2_x)
        df["PhaseDelay_y"]=alfa*(df.L1_y-df.L2_y)
        df["CodeDelay_x"]=alfa*(df.C2_x-df.C1_x)
        df["CodeDelay_y"]=alfa*(df.C2_y-df.C1_y)
        #add column with indicators of time separation
        df=label_arcs(df)
        for arc in df.ARCS.values:
            #Search for cycle Slips
            slips1=cycle_slips(df.PhaseDelay_x.as_matrix(),2.5)
            print slips1
            slips2=cycle_slips(df.PhaseDelay_y.as_matrix(),2.5)
    
       
    break
    

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


TypeError: only integer arrays with one element can be converted to an index

In [ ]:
print df3.shape
print df.shape

In [ ]:
print "Satelites ibservados ",np.unique(df3.PRN.values).size
np.unique(df3.PRN.values)

In [ ]:
#df.head(4)
plt.scatter(df.TIME,df.PhaseDelay_x)
df.TIME.size

In [ ]:
df